<center><strong>Sistem Temu Kembali Informasi</strong><br />
</center>

<strong>Outline pertemuan:</strong><br />
<li> TFIDF</li>
<li> BM-25</li>

# TF-IDF


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
query='first time'
query_vec= vectorizer.transform([query])
results=cosine_similarity(X, query_vec).reshape((-1))
print (results)
for i in results.argsort()[::-1]:
    print(corpus[i])

[0.58028582 0.         0.         0.58028582]
Is this the first document?
This is the first document.
And this is the third one.
This document is the second document.


In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

corpus = [
    'Budi bermain bola ketika ibu ke pasar',
    'ayah mencuci mobil digarasi dan menggoda ibu ibu',
    'ibu membeli sayur dan buah buahan',
    'kakak sedang tidur di kamar',
]
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
query='ibu'
query_vec= vectorizer.transform([query])
results=cosine_similarity(X, query_vec).reshape((-1))
print (results)
for i in results.argsort()[::-1]:
    print(corpus[i])

[0.25215917 0.47406737 0.28462634 0.        ]
ayah mencuci mobil digarasi dan menggoda ibu ibu
ibu membeli sayur dan buah buahan
Budi bermain bola ketika ibu ke pasar
kakak sedang tidur di kamar


<h2>BM-25 Rank Algorithm</h2>

In [3]:
from rank_bm25 import BM25Okapi

corpus = [
    "Hello there good man!",
    "It is quite windy in London",
    "How is the weather today?"
]

tokenized_corpus = [doc.split(" ") for doc in corpus]
print (tokenized_corpus)
bm25 = BM25Okapi(tokenized_corpus)
query = "good man"
tokenized_query = query.split(" ")
print (tokenized_query)


doc_scores = bm25.get_scores(tokenized_query)
bm25.get_top_n(tokenized_query, corpus, n=1)
print (bm25.get_top_n(tokenized_query, corpus, n=1))

[['Hello', 'there', 'good', 'man!'], ['It', 'is', 'quite', 'windy', 'in', 'London'], ['How', 'is', 'the', 'weather', 'today?']]
['good', 'man']
['Hello there good man!']


In [4]:
from rank_bm25 import BM25Okapi

corpus = [
    'Budi bermain bola ketika ibu ke pasar',
    'ayah mencuci mobil digarasi dan menggoda ibu ibu',
    'ibu membeli sayur dan buah buahan',
    'kakak sedang tidur di kamar',
]

tokenized_corpus = [doc.split(" ") for doc in corpus]
print (tokenized_corpus)
bm25 = BM25Okapi(tokenized_corpus)
query = "ibu"
tokenized_query = query.split(" ")
print (tokenized_query)


doc_scores = bm25.get_scores(tokenized_query)
bm25.get_top_n(tokenized_query, corpus, n=1)
print (bm25.get_top_n(tokenized_query, corpus, n=1))

[['Budi', 'bermain', 'bola', 'ketika', 'ibu', 'ke', 'pasar'], ['ayah', 'mencuci', 'mobil', 'digarasi', 'dan', 'menggoda', 'ibu', 'ibu'], ['ibu', 'membeli', 'sayur', 'dan', 'buah', 'buahan'], ['kakak', 'sedang', 'tidur', 'di', 'kamar']]
['ibu']
['ayah mencuci mobil digarasi dan menggoda ibu ibu']


## Semantic Search

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\M-SI\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
from sentence_transformers import SentenceTransformer

# New corpus
corpus = [
    "Mauris lacinia magna a purus dignissim, ac bibendum metus gravida",
    "Mauris at sapien consectetur tellus feugiat convallis",
    "Mauris accumsan arcu ut turpis laoreet pretium",
    "Maecenas et mauris volutpat, scelerisque ante quis, iaculis metus",
    "Vivamus et felis ut ipsum aliquam fringilla",
]

# Tokenize the sentences to get a list of words
words = [word for sentence in corpus for word in nltk.word_tokenize(sentence)]

# Choose a random word from the corpus as the query
query = "mauris"

# Load pre-trained sentence transformer model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Encode corpus and query
document_embeddings = model.encode(corpus)
query_embedding = model.encode([query])

# Compute cosine similarity between query and corpus
cosine_similarities = cosine_similarity(query_embedding, document_embeddings)

# Get index of the most relevant document
most_relevant_doc_index = cosine_similarities.argmax()

# print all documents along with their cosine similarity score and sort them
num = 1
for i in cosine_similarities.argsort()[0][::-1]:
    print(num, corpus[i], cosine_similarities[0][i])
    num += 1

1 Mauris accumsan arcu ut turpis laoreet pretium 0.5366721
2 Mauris at sapien consectetur tellus feugiat convallis 0.5067044
3 Mauris lacinia magna a purus dignissim, ac bibendum metus gravida 0.391652
4 Maecenas et mauris volutpat, scelerisque ante quis, iaculis metus 0.36695278
5 Vivamus et felis ut ipsum aliquam fringilla 0.28199077


## Latent Semantic Indexing (LSI)

In [7]:
from gensim import corpora, models, similarities

corpus = [
    "Sed ullamcorper orci in odio suscipit, ac ultricies tortor iaculis.",
    "Cras vitae velit eu massa interdum varius.",
    "Suspendisse mollis nisi vel enim maximus mollis.",
    "Quisque sit amet tortor hendrerit, commodo odio ut, posuere turpis.",
    "Praesent vitae odio laoreet, aliquet diam in, hendrerit sapien."
]

# Tokenize the corpus
tokenized_corpus = [doc.split(" ") for doc in corpus]

# Create a dictionary representation of the documents
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the corpus into Bag of Words using dictionary
bow_corpus = [dictionary.doc2bow(text) for text in tokenized_corpus]

# Create an LSI transformation of the corpus
lsi = models.LsiModel(bow_corpus, id2word=dictionary, num_topics=2)

# Create a similarity index
index = similarities.MatrixSimilarity(lsi[bow_corpus])

# Query transformation
query = 'in'
query_bow = dictionary.doc2bow(query.split())
query_lsi = lsi[query_bow]

# Perform a similarity query against the corpus
sims = index[query_lsi]

# print all documents along with their similarity score and sort them
num = 1
for i in sims.argsort()[::-1]:
    print(num, corpus[i], sims[i])
    num += 1

1 Quisque sit amet tortor hendrerit, commodo odio ut, posuere turpis. 0.9926931
2 Sed ullamcorper orci in odio suscipit, ac ultricies tortor iaculis. 0.9926931
3 Praesent vitae odio laoreet, aliquet diam in, hendrerit sapien. 0.05814138
4 Suspendisse mollis nisi vel enim maximus mollis. 0.0
5 Cras vitae velit eu massa interdum varius. -0.2601207


## TF Cosine

In [8]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample corpus
corpus = [
    'Aliquam interdum odio suscipit, vulputate nulla sit amet, ultricies purus.',
    'Nulla at odio at odio pharetra tempus et ut lacus.',
    'Integer hendrerit urna a orci vulputate, sed pharetra enim vehicula.',
    'Sed tempus sapien sit amet est luctus, ac tristique metus ultrices.',
]

# Query
query = "odio"

# Initialize CountVectorizer with binary=False
vectorizer = CountVectorizer(binary=False)

# Fit and transform the corpus
X = vectorizer.fit_transform(corpus)

# Transform the query
query_vec = vectorizer.transform([query])

# Compute cosine similarity between query and corpus
cosine_similarities = cosine_similarity(query_vec, X).flatten()

# print all documents along with their cosine similarity score and sort them
num = 1
for i in cosine_similarities.argsort()[::-1]:
    print(num, corpus[i], cosine_similarities[i])
    num += 1


1 Nulla at odio at odio pharetra tempus et ut lacus. 0.5345224838248488
2 Aliquam interdum odio suscipit, vulputate nulla sit amet, ultricies purus. 0.31622776601683794
3 Sed tempus sapien sit amet est luctus, ac tristique metus ultrices. 0.0
4 Integer hendrerit urna a orci vulputate, sed pharetra enim vehicula. 0.0


## Boolean Retrieval

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

# Sample corpus
corpus = [
    "Sed interdum nisi ultrices, ornare tellus nec, laoreet tortor.",
    "Donec maximus nibh at mi suscipit, tempor finibus purus rutrum",
    "Sed gravida erat a facilisis ornare",
    "Sed pharetra urna sed ante ornare suscipit",
    "Duis dignissim urna in scelerisque gravida"
]

# Initialize CountVectorizer with binary=True
vectorizer = CountVectorizer(binary=True)

# Fit and transform the corpus
X = vectorizer.fit_transform(corpus)

# Convert the result to an array
binary_term_document_matrix = X.toarray()

# Now you can use this binary_term_document_matrix for Boolean Retrieval
# For example, to find documents that contain both 'document' and 'first', you can do:
document_index = vectorizer.vocabulary_.get('sed')
first_index = vectorizer.vocabulary_.get('ornare')

# Use logical AND to find documents that contain 'document' and 'first'
result = binary_term_document_matrix[:, document_index] & binary_term_document_matrix[:, first_index]

# Print the result
print(result)

# print the document that contain both 'document' and 'first'
for i, doc in enumerate(corpus):
    if result[i]:
        print(doc)

[1 0 1 1 0]
Sed interdum nisi ultrices, ornare tellus nec, laoreet tortor.
Sed gravida erat a facilisis ornare
Sed pharetra urna sed ante ornare suscipit


## IDF Cosine

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
corpus = [
    'Integer hendrerit urna a orci vulputate, sed pharetra enim vehicula',
    'Duis dignissim urna in scelerisque gravida',
    'Phasellus vel urna aliquet, ultricies libero eu, tincidunt diam',
    'Donec maximus nibh at mi suscipit, tempor finibus purus rutrum'
]

query = 'urna'

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer(use_idf=True, norm=None)

# Fit and transform the corpus
X_corpus = vectorizer.fit_transform(corpus)

# Transform the query
X_query = vectorizer.transform([query])

# Compute the cosine similarity
cosine_similarities = cosine_similarity(X_query, X_corpus)

# Print the cosine similarities sort them
num = 1
for i in cosine_similarities.argsort()[0][::-1]:
    print(num, corpus[i], cosine_similarities[0][i])
    num += 1

1 Duis dignissim urna in scelerisque gravida 0.27448673838643983
2 Phasellus vel urna aliquet, ultricies libero eu, tincidunt diam 0.2201328763012063
3 Integer hendrerit urna a orci vulputate, sed pharetra enim vehicula 0.2201328763012063
4 Donec maximus nibh at mi suscipit, tempor finibus purus rutrum 0.0


# Note: corpus text source was from lorem ipsum ^_^